In [1]:
import numpy as np
import pandas as pd
import datetime as dt

data = pd.read_csv("data.csv")
data_new=data.rename(columns={"TermId": "ATM ID"})
# print(data_new.dtypes["ATM ID"])
# data_new["ATM ID"]=data_new["ATM ID"].astype(float)
# print(data_new.columns)
print(data_new.shape)

ticket = pd.read_csv("maintenance.csv")
ticket.columns = ["Ticket ID", "ATM ID", "Ticket start", "Ticket stop", "Fault", "Activity comment"]
# ticket = ticket.reindex(columns=columns)
# print(ticket.dtypes["ATM ID"])
ticket["ATM ID"]=ticket["ATM ID"].astype(object)
# print(ticket.columns)
print(ticket.shape)



(984245, 5)
(25939, 6)


In [2]:
data_new["FaultStartTime"] = pd.to_datetime(data_new["FaultStartTime"], format='%Y-%m-%d %H:%M:%S.%f')
data_new["FaultEndTime"] = pd.to_datetime(data_new["FaultEndTime"], format='%Y-%m-%d %H:%M:%S.%f')

ticket["Ticket start"] = pd.to_datetime(ticket["Ticket start"], format='%d-%m-%Y %H:%M')
ticket["Ticket stop"] = pd.to_datetime(ticket["Ticket stop"], format='%d-%m-%Y %H:%M')

data_new = data_new.sort_values(by = ["FaultStartTime"], ignore_index = True)
ticket = ticket.sort_values(by = ["Ticket start"], ignore_index = True)

data_new = data_new[data_new["FaultStartTime"] >= dt.datetime(2020, 8, 1, 0, 0, 0)]
data_new = data_new[data_new["FaultStartTime"] < dt.datetime(2020, 9, 1, 0, 0, 0)]
data_new.reset_index()

,index,ATM ID,FaultStartTime,FaultEndTime,FaultDesc,age
0,119989,1FNVAR81,2020-08-01 00:00:07.980,2020-08-01 07:49:23.417,security camera fatal (NCR),469
1,119990,1FNVAR85,2020-08-01 00:00:07.983,2020-08-01 13:09:49.577,security camera fatal (NCR),789
2,119991,1FWRGH40,2020-08-01 00:00:16.733,2020-08-01 00:07:44.237,down - communication failure (NCR),7
3,119992,1RDDMUM81,2020-08-01 00:00:32.350,2020-08-01 00:07:59.477,down - communication failure (DIEBOLD 10xx),7
4,119993,1RDNAHM18,2020-08-01 00:00:39.037,2020-08-04 13:51:59.843,security camera fatal (NCR),5151
...,...,...,...,...,...,...
530325,650314,1FNKAG01,2020-08-31 23:58:03.467,2020-09-01 01:07:42.960,down - communication failure (NCR),69
530326,650315,1RDDMUM03,2020-08-31 23:59:24.630,2020-09-01 16:37:02.310,cassette 3 faulted (DIEBOLD 10xx),998
530327,650316,1RDDMUM03,2020-08-31 23:59:24.630,2020-09-01 16:37:02.310,All Cassette Faulted (DIEBOLD),998
530328,650317,1RDDMUM89,2020-08-31 23:59:27.343,2020-09-01 00:10:37.137,Close No Other Fault (DIEBOLD),11


In [3]:
fnt=pd.merge(data_new,ticket,how="inner",on="ATM ID") # the 2 sheets merged, consist of all errors and activity
print(fnt.shape)
# print(fnt)



(4443150, 10)


In [4]:
fnt_SLM = fnt[fnt["Activity comment"] == "SLM"] # consist of all errors but only SLM activity
print(fnt_SLM.shape)
# print(fnt_SLM.columns)
# print(fnt_SLM)



(340730, 10)


In [5]:
priority1errors = ["All Cassette Faulted (DIEBOLD)",
                   "All Cassette Faulted (NCR)",
                   "both supply and supervisor switches on (DIEBOLD 10xx)",
                   "card reader failure (DIEBOLD 10xx)",
                   "cash handler  fatal (NCR)",
                   "cash handler reject bin overfill (NCR)",
                   "Cashout (DIEBOLD)",
                   "Cashout (NCR)",
                   "close (DIEBOLD 10xx)",
                   "close (NCR)",
                   "down - communication failure (DIEBOLD 10xx)",
                   "down - communication failure (NCR)",
                   "encrypted pin pad failure (DIEBOLD 10xx)",
                   "encryptor  fatal (NCR)",
                   "journal paper out (NCR)",
                   "journal printer  fatal (NCR)",
                   "journal printer paper out (DIEBOLD 10xx)",
                   "magnetic card read/write  fatal (NCR)",
                   "supervisor mode alarm is on (NCR)",
                   "supervisor switch on (DIEBOLD 10xx)",
                   "receipt printer fatal (NCR)"]
# final=fnt_SLM.replace(priority1errors,"priority1errors")
# final=final[final["FaultDesc"] == priority1errors]
final=fnt_SLM[fnt_SLM["FaultDesc"].isin(priority1errors)]  # consist of Priority 1 errors with SLM activity
final.to_csv("final.csv")
print(final.shape)
# print(final)



(154187, 10)


In [20]:
# atm_id = {}
# atm = final["ATM ID"].unique() # to check on final or on fnt ?
# for i in atm:
#     atm_id[i] = final[final["ATM ID"] == i][['FaultStartTime', 'FaultEndTime', 'FaultDesc']].values.tolist()
# print(len(atm_id))
# atm=final["ATM ID"].unique()
# print(len(atm))
atm=final["ATM ID"].unique().tolist()
print(len(atm))
print(atm[0])

1190
1FNVAR85


In [21]:
# errors=final["FaultDesc"].unique()
# T=len(errors) # number of error types
# print(T)

# right now, the domain of error types is the priority 1 errors
# we also assume that if activity==SLM, then only the instance is +ve, else it is -ve
l1=len(atm)
l2=len(priority1errors)

error_count_pstv = [[0 for j in range(l2)] for i in range(l1)]
error_count_tot = [[0 for j in range(l2)] for i in range(l1)]
confidence_value = [[0 for j in range(l2)] for i in range(l1)]

error_count_pstv_9_18=[0 for i in range(l1)]
error_count_tot_9_18=[0 for i in range(l1)]
confidence_value_9_18=[0 for i in range(l1)]

targets = {m : [] for m in atm}

In [22]:
fnt_FLM_SLM=fnt[(fnt["Activity comment"] == "SLM") | (fnt["Activity comment"] == "FLM")]
final_FS=fnt_FLM_SLM[fnt_FLM_SLM["FaultDesc"].isin(priority1errors)]
final_FS.to_csv("final_FS.csv")
print(final_FS.shape)
atm_FS=final_FS["ATM ID"].unique().tolist()
l_FS=len(atm_FS)
print(l_FS)
num_tickets = [0 for i in range(l_FS)]
for i in range(l_FS):
#     print(i)
    atm_val=atm_FS[i]
    sub_data=final_FS[final_FS["ATM ID"]==atm_val]
    num_tickets[i]=len(sub_data["Ticket ID"].unique())
    print(i,num_tickets[i])
# num_tickets.value_counts(ascending=True)
# print(num_tickets)
# Counter(num_tickets)

(1587898, 10)
3568
0 2
1 8
2 10
3 3
4 4
5 1
6 4
7 2
8 6
9 16
10 10
11 3
12 8
13 7
14 3
15 1
16 2
17 2
18 3
19 14
20 4
21 7
22 10
23 4
24 3
25 1
26 5
27 13
28 4
29 18
30 2
31 8
32 15
33 3
34 4
35 3
36 3
37 3
38 8
39 2
40 2
41 1
42 4
43 6
44 1
45 3
46 5
47 6
48 4
49 1
50 2
51 3
52 1
53 2
54 13
55 1
56 9
57 2
58 1
59 3
60 4
61 2
62 4
63 4
64 3
65 3
66 8
67 13
68 3
69 3
70 2
71 4
72 3
73 2
74 2
75 3
76 3
77 4
78 12
79 13
80 1
81 2
82 15
83 5
84 2
85 15
86 4
87 1
88 3
89 2
90 4
91 2
92 4
93 3
94 2
95 2
96 2
97 1
98 6
99 7
100 2
101 2
102 3
103 2
104 12
105 3
106 2
107 6
108 3
109 1
110 1
111 3
112 10
113 6
114 7
115 6
116 6
117 1
118 15
119 5
120 1
121 10
122 2
123 12
124 7
125 4
126 4
127 3
128 1
129 1
130 1
131 6
132 7
133 5
134 14
135 6
136 6
137 4
138 1
139 7
140 7
141 3
142 2
143 5
144 7
145 15
146 3
147 14
148 5
149 5
150 5
151 6
152 2
153 5
154 5
155 3
156 6
157 6
158 5
159 5
160 4
161 3
162 4
163 3
164 3
165 10
166 3
167 8
168 1
169 2
170 7
171 1
172 3
173 1
174 3
175 2
176 20
177 4

In [25]:
from collections import Counter
v=Counter(num_tickets)
sorted(v.keys())
print(v)

Counter({2: 632, 1: 594, 3: 529, 4: 431, 5: 343, 6: 254, 7: 176, 8: 130, 9: 88, 10: 82, 12: 64, 11: 58, 14: 35, 13: 34, 15: 25, 17: 18, 16: 16, 18: 15, 19: 13, 23: 7, 20: 5, 21: 5, 22: 5, 24: 5, 27: 2, 28: 1, 25: 1})


In [8]:
# for atm_val in atm: # iter1 (over ATMs)
dataset=pd.DataFrame()
dataset1=pd.DataFrame()
for i in range(l1): # iter1 (over ATMs)
    print(i)
    atm_val=atm[i]
    sub_data=final[final["ATM ID"]==atm_val]
    month_start = dt.datetime(2020, 8, 1, 0, 0, 0) # to generalize this ...
    month_end = dt.datetime(2020, 8, 31, 23, 59, 59) # to generalize this ...
    x = 2 # number of subwindows # to generalize this ...
    m = 10 # length of each subwindow # to generalize this ...
    y = 6 # length of prediction window # to generalize this ...
#     subwindow_len = dt.timedelta(days = m, hours = 0, minutes = 0, seconds = 0)  # length of each measurement unit
    ow_gap = dt.timedelta(days = 2, hours = 0, minutes = 0, seconds = 0)  # gap between two successive observation windows
    ow_start = month_start
    ow_end = ow_start + dt.timedelta(days = x*m, hours = 0, minutes = 0, seconds = 0)
    pred_point = ow_start + dt.timedelta(days = x*m, hours = 0, minutes = 0, seconds = 0) # buffer window not considered now
    pred_end = pred_point + dt.timedelta(days = y, hours = 0, minutes = 0, seconds = 0)
    
    num_obs=0
    
    while(pred_end<=month_end): # iter2 (over each observation window)
        num_obs+=1
        s=sub_data
        s=s[s["FaultStartTime"]>=ow_start]
        s=s[s["FaultEndTime"]<ow_end]
        s=s.reset_index(drop=True)
        ispositive = False
        ticket_checked=False
        all_tickets=s["Ticket ID"].unique()
        for ticket in all_tickets:
            for index, row in s.iterrows():
                if(row["Ticket ID"]==ticket):
                    ticket_checked=True
                    if(row["Ticket start"]>=pred_point and row["Ticket stop"]<pred_end):
                        ispositive=True
                        break
                if(ticket_checked==True):
                    break
            if(ispositive==True):
                break
            ticket_checked=False
            
        targets[atm_val].append(0 if ispositive == False else 1)
        
#         for error in priority1errors:
        for j in range(l2):
            error=priority1errors[j]
            error_count = s[s["FaultDesc"]==error].shape[0]
#             error_count_tot[atm_val][error] += error_count
            error_count_tot[i][j] += error_count
            if(ispositive==True):
#                 error_count_pstv[atm_val][error] += error_count
                error_count_pstv[i][j] += error_count
        
#         index1=1
#         index2=2
#         s_l=s.shape[0]
#         while(index2<s_l):
#             r1=s[index1]
#             r2=s[index2]
#             b1=r1["FaultDesc"]==priority1errors[9] and r2["FaultDesc"]==priority1errors[18]
#             b2=r2["FaultDesc"]==priority1errors[9] and r1["FaultDesc"]==priority1errors[18]
#             if(b1 or b2):
#                 error_count_tot_9_18+=1
#                 if(ispositive==True):
#                     error_count_pstv_9_18+=1
#             index1+=2
#             index2+=2
            
        index1=1
        index2=2
        s_l=s.shape[0]
#         for index,row in s.iterrows():
        for index in s.index:
#             print(index)
            if(index1==index and index2!=s_l):
#                 r1=row
#                 r2=r1.shift(-1) # issue here
                b1=s["FaultDesc"][index]==priority1errors[9] and s["FaultDesc"][index+1]==priority1errors[18]
                b2=s["FaultDesc"][index]==priority1errors[18] and s["FaultDesc"][index+1]==priority1errors[9]
#                 print(r1)
#                 print(r2)
#                 print(index,index1,index2,b1,b2)
                if(b1 or b2):
                    error_count_tot_9_18[i]+=1
                    if(ispositive==True):
                        error_count_pstv_9_18[i]+=1
                index1+=2
                index2+=2
            
#         for index, row in s.iterrows():
#             check if (row["FaultDesc"]==priority1errors[9] and nextrow["FaultDesc"]==priority1errors[18]) 
#             or (row["FaultDesc"]==priority1errors[18] and nextrow["FaultDesc"]==priority1errors[9])
#             error_count_tot_9_18+=val
#             if(ispositive==True):
#                 error_count_pstv_9_18+=val
                
        ow_start += ow_gap
        ow_end = ow_start + dt.timedelta(days = x*m, hours = 0, minutes = 0, seconds = 0)
        pred_point = ow_start + dt.timedelta(days = x*m, hours = 0, minutes = 0, seconds = 0) # buffer window not considered now
        pred_end = pred_point + dt.timedelta(days = y, hours = 0, minutes = 0, seconds = 0)
    
#     for error in priority1errors:
    for j in range(l2):
#         confidence_value[atm_val][error] = error_count_pstv[atm_val][error]/error_count_tot[atm_val][error]
#         print(confidence_value[atm_val][error])
        if(error_count_tot[i][j]!=0):
            confidence_value[i][j] = error_count_pstv[i][j]/error_count_tot[i][j]
    
    print(error_count_tot_9_18[i])
    if(error_count_tot_9_18[i]!=0):
        confidence_value_9_18[i]=error_count_pstv_9_18[i]/error_count_tot_9_18[i]
    print(confidence_value_9_18[i])

    profile = pd.DataFrame([atm_val for itr in range(num_obs)], columns = ["ATM ID"])
    target_df = pd.DataFrame(targets[atm_val], columns = ["Target"])
    df = pd.concat([profile, target_df], axis = 1)
    dataset = pd.concat([dataset, df])
    
    profile1 = pd.DataFrame([atm_val for itr in range(l2)], columns = ["ATM ID"])
    error1 = pd.DataFrame(priority1errors, columns = ["Error Name"])
    confidence1 = pd.DataFrame(confidence_value[i], columns = ["Confidence Value"])
    df1 = pd.concat([profile1, error1, confidence1], axis = 1)
    dataset1 = pd.concat([dataset1, df1])

0
22
0.6818181818181818
1
31
0.0
2
30
0.36666666666666664
3
3
0.0
4
12
0.0
5
15
0.3333333333333333
6
39
0.0
7
7
0.0
8
4
0.0
9
47
0.6382978723404256
10
3
0.3333333333333333
11
24
1.0
12
19
0.0
13
12
0.6666666666666666
14
42
0.0
15
29
0.0
16
12
0.0
17
15
0.0
18
6
0.0
19
11
0.0
20
12
0.0
21
8
0.875
22
5
0.0
23
6
0.0
24
26
0.0
25
13
1.0
26
18
0.0
27
11
0.2727272727272727
28
37
0.0
29
14
0.0
30
0
0
31
10
0.0
32
14
0.7142857142857143
33
0
0
34
5
0.0
35
17
0.6470588235294118
36
1
0.0
37
9
0.0
38
0
0
39
20
0.0
40
34
0.29411764705882354
41
10
0.0
42
11
0.0
43
21
0.0
44
20
0.0
45
32
0.0
46
11
0.0
47
27
0.0
48
17
0.0
49
23
1.0
50
10
0.0
51
9
0.6666666666666666
52
9
0.0
53
23
0.391304347826087
54
12
0.25
55
20
1.0
56
34
0.29411764705882354
57
46
0.6304347826086957
58
15
0.0
59
29
0.0
60
2
1.0
61
8
0.25
62
10
0.3
63
15
0.0
64
8
0.375
65
34
0.35294117647058826
66
2
0.0
67
25
0.0
68
31
0.0
69
9
1.0
70
29
0.0
71
10
0.0
72
25
0.64
73
3
0.0
74
20
0.0
75
13
0.0
76
11
0.0
77
18
0.0
78
2
1.0
79
22
0.681818

In [13]:
# print(confidence_value_9_18[0])
Error_sigf={j: False for j in range(l2)}
for j in range(l2):
    for i in range(l1):
        Error_sigf[j] = Error_sigf[j] or (confidence_value[i][j]!=0)
        if(Error_sigf[j]==True):
#             print(i)
            break
# print(Error_sigf)

0


In [33]:
print(dataset)

       ATM ID  Target
0    1FNVAR85       0
1    1FNVAR85       1
2    1FNVAR85       1
0   1RDNAJM02       0
1   1RDNAJM02       0
..        ...     ...
1    1FNSRO07       0
2    1FNSRO07       0
0    1FNMHB03       0
1    1FNMHB03       0
2    1FNMHB03       0

[3570 rows x 2 columns]


In [34]:
print(dataset1)

      ATM ID                                         Error Name  \
0   1FNVAR85                     All Cassette Faulted (DIEBOLD)   
1   1FNVAR85                         All Cassette Faulted (NCR)   
2   1FNVAR85  both supply and supervisor switches on (DIEBOL...   
3   1FNVAR85                 card reader failure (DIEBOLD 10xx)   
4   1FNVAR85                          cash handler  fatal (NCR)   
..       ...                                                ...   
16  1FNMHB03           journal printer paper out (DIEBOLD 10xx)   
17  1FNMHB03              magnetic card read/write  fatal (NCR)   
18  1FNMHB03                  supervisor mode alarm is on (NCR)   
19  1FNMHB03                supervisor switch on (DIEBOLD 10xx)   
20  1FNMHB03                        receipt printer fatal (NCR)   

    Confidence Value  
0                0.0  
1                0.0  
2                0.0  
3                0.0  
4                0.0  
..               ...  
16               0.0  
17         

In [35]:
# from collections import Counter
Counter(dataset.Target)

Counter({0: 2801, 1: 769})

In [36]:
dataset.to_csv("Feature.csv", index=False)
dataset1.to_csv("Confidence.csv", index=False)